# 난수 생성

해당 절에서는 `jax.random`과 유사난수(pseudo random number generation, PRNG)에 대해 알아보겠습니다.

컴퓨터는 순수한 난수를 생성할 수 없습니다. 때문에 컴퓨터로 하여금 난수를 생성하게끔 하기 위해서 **유사난수** 라는 방법이 생겨났습니다. 순수한 난수는 아니지만, 사람이 사용하는 데에는 충분히 불확실성을 보여주는 것을 목표로 합니다.

유사난수는 초기값에 기반하여 난수를 생성하게 되고, 이 초기값을 `seed`라고 합니다. 이후로 줄줄이 나오게 되는 난수들은, 이전에 난수를 생성할  때 같이 생성된 어떠한 상태(`state`)에 기반하여 생성된 값입니다. 즉, 이전 상태가 같으면 같은 난수를 생성하게 됩니다.

난수 생성은 현대 머신러닝에 필수적인 존재입니다. JAX는 최대한 많은 부분을 numpy와 호환되도록 설계하고자 노력했지만, 난수 생성은 numpy와 전혀 다른 메커니즘을 가진 몇없는 기능 중 하나 입니다. JAX 개발자들이 굳이 난수 부분을 numpy 와 다르게 만든 데에는 numpy의 난수 생성이 현대 머신러닝에 맞지 않는 부분이 있기 때문이라고 유추할 수 있습니다.

NumPy와 JAX의 난수생성의 차이점을 더 잘 이해하기 위해서 먼저 NumPy가 난수를 어떻게 생성하는지부터 살펴보겠습니다.

## NumPy의 난수 생성

NumPy에서는 `numpy.random` 함수를 통해 유사난수를 생성할 수 있습니다. NumPy는 전역변수 `state`에 기반하여 유사난수를 생성하고, 이 `state`는 `numpy.random.seed` 함수를 통해 설정됩니다.

In [1]:
import numpy as np
np.random.seed(0)

아래 코드와 같이 `seed`를 통해 결정된 `state`를 확인할 수 있습니다:

In [2]:
def print_truncated_random_state():
  """To avoid spamming the outputs, print only part of the state."""
  full_random_state = np.random.get_state()
  print(str(full_random_state)[:460], '...')

print_truncated_random_state()

('MT19937', array([         0,          1, 1812433255, 1900727105, 1208447044,
       2481403966, 4042607538,  337614300, 3232553940, 1018809052,
       3202401494, 1775180719, 3192392114,  594215549,  184016991,
        829906058,  610491522, 3879932251, 3139825610,  297902587,
       4075895579, 2943625357, 3530655617, 1423771745, 2135928312,
       2891506774, 1066338622,  135451537,  933040465, 2759011858,
       2273819758, 3545703099, 2516396728, 127 ...


`state`는 난수를 생성할 때마다 업데이트됩니다:

In [3]:
np.random.seed(0)
print_truncated_random_state()

('MT19937', array([         0,          1, 1812433255, 1900727105, 1208447044,
       2481403966, 4042607538,  337614300, 3232553940, 1018809052,
       3202401494, 1775180719, 3192392114,  594215549,  184016991,
        829906058,  610491522, 3879932251, 3139825610,  297902587,
       4075895579, 2943625357, 3530655617, 1423771745, 2135928312,
       2891506774, 1066338622,  135451537,  933040465, 2759011858,
       2273819758, 3545703099, 2516396728, 127 ...


In [4]:
_ = np.random.uniform()
print_truncated_random_state()

('MT19937', array([2443250962, 1093594115, 1878467924, 2709361018, 1101979660,
       3904844661,  676747479, 2085143622, 1056793272, 3812477442,
       2168787041,  275552121, 2696932952, 3432054210, 1657102335,
       3518946594,  962584079, 1051271004, 3806145045, 1414436097,
       2032348584, 1661738718, 1116708477, 2562755208, 3176189976,
        696824676, 2399811678, 3992505346,  569184356, 2626558620,
        136797809, 4273176064,  296167901, 343 ...


NumPy를 통해서 스칼라 값과 배열 모두 난수를 생성할 수 있습니다. 3의 길이를 가진 정규분포에 따른 난수를 다음과 같이 생성할 수 있습니다:

In [5]:
np.random.seed(0)
print(np.random.uniform(size=3))

[0.5488135  0.71518937 0.60276338]


NumPy는 *sequential equivalent guarantee*라는 것을 제공하는데, 이는 같은 `seed`를 가진다면 스칼라 난수를을 따로 3번을 생성하던, 배열로 3개의 난수를 동시에 생성하던 같은 값을 생성한다는 것을 말합니다:

In [6]:
np.random.seed(0)
print("individually:", np.stack([np.random.uniform() for _ in range(3)]))

np.random.seed(0)
print("all at once: ", np.random.uniform(size=3))

individually: [0.5488135  0.71518937 0.60276338]
all at once:  [0.5488135  0.71518937 0.60276338]


## JAX의 난수 생성

NumPy의 난수 생성 방식은 여러가지 한계점이 존재하는데, JAX는 이러한 문제를 해결하고자 NumPy와는 완전히 다른 난수 방식을 택하게 되었습니다. Jax는 **재현 가능하고, 병렬화 가능하며, 벡터화 가능한 난수 생성**을 목표로 합니다.

1. **재현성(Reproducibility)** <br/>
2. **병렬화(Parallelism)** <br/>
3. **벡터화(Vectorization)**

왜 그런지는 이후 설명하도록 하고, 일단은 기본적인 JAX의 난수 생성 방법을 알아보겠습니다. 먼저 아래 NumPy의 예시를 보겠습니다:

In [8]:
import numpy as np

np.random.seed(0)

def bar():
    return np.random.uniform()
def baz():
    return np.random.uniform()

def foo():
    return bar() + 2 * baz()

print(foo())

1.9791922366721637


함수 `foo`는 함수 `bar`과 `baz`로부터 난수를 받아와서 계산을 한 결과값을 리턴합니다.

위 코드의 최종 결과값이 **1. 재현성**을 충족하려면 코드가 반드시 순차적으로 실행되어야 합니다. 너무 당연한 소리같이 들릴것이, 코드는 당연히 순차적으로 실행되기 때문입니다. 하지만 JAX에서는 그렇지 않습니다. JAX는 더욱 높은 성능을 위해 함수들을 JIT 컴파일하게 되고, 이때 컴파일된 JAX 코드들은 파이썬의 순서 규칙과는 무관하게 JAX만의 규칙 하에 돌아가게 되고, 이는 **1. 재현성**에 위배될 수 있는 여지를 만듭니다. 또한, 다중GPU 작업이나 병렬컴퓨팅을 하게 될 경우 전역변수 `state` 를 사용하여 난수를 생성하는 NumPy 방식의 경우 효과적인 병렬화를 방해하게 됩니다.

### 명시적 난수 상태(state)

위와 같은 문제를 해결하기 위해 JAX는 `key`를 통해 매우 명시적으로 `state`를 관리하게 됩니다.

In [14]:
from jax import random

key = random.key(42)
print(key)

Array((), dtype=key<fry>) overlaying:
[ 0 42]


key는 `uint32`를 기반으로 한, PRNG 타입을 가진 배열입니다.

key는 위 **NumPy의 난수 생성** 파트에서 살펴본 NumPy의 `state`와 같은 역할을 하지만 JAX에서는 이 key를 random 함수에 매번 직접 인자로 넣어주는 방식으로 명시적으로 관리하게 됩니다. 중요한 점은, JAX의 random 함수에 key를 넣어주게 되면 random 함수는 key를 **소비**하게 됩니다. 해당 key를 다시 사용하는 것은 JAX에서 권장하는 방법이 아니며, 만약 같은 key를 재사용하게 되면 똑같은 난수를 생성하게 됩니다.

In [11]:
print(random.normal(key))
print(random.normal(key))

-0.18471177
-0.18471177


다른 random 함수더라도 같은 key를 재사용하게 되면 두 결과값에 종속성이 생길 수 있습니다.

그러니 **동일한 난수를 생성하고자 하는게 아니라면, 절대 키를 재사용하지 마세요.**

또 다른 난수를 생성하기 위해서는 `split()` 함수를 통해 새로운 key를 생성하고, 이를 사용하면 됩니다:

In [15]:
for i in range(3):
  new_key, subkey = random.split(key) # key는 new_key와 subkey를 생성하는 과정에서 소비되었으며, 재사용하지 않습니다.

  val = random.normal(subkey)
  del subkey  # subkey는 random.normal() 에 의해 소비됨.
  print(f"draw {i}: {val}")
  key = new_key  # new_key는 사용되지 않았으므로 이후 사용 가능.

draw 0: 1.369469404220581
draw 1: -0.19947023689746857
draw 2: -2.298278331756592


`jax.random.split()` 함수는 하나의 key를 받아서 여러개의 key를 반환하는 함수입니다. 반환된 key 중 하나는 보관하고, 나머지 key들은 다른 난수를 생성하는데 사용하게 됩니다(위 코드에서 보관되는 key는 `new_key`, 사용되는 key는 `subkey`가 됩니다). 그리고 사용된 키는 **절대** 재사용하지 않습니다. 새로운 난수를 생성해야 한다면 다시 `jax.random.split()`을 통해 새로운 key를 발급받은 후 사용하면 됩니다. **가장 중요한 부분은 key를 재사용하지 않는 것입니다.**